In [2]:
from pyod.models.knn import KNN 
import numpy as np
import pandas as pd

In [3]:
from pyod.utils.data import generate_data

In [4]:
from pyod.utils.data import evaluate_print

In [5]:
from utils import UtilsKy

In [11]:
#from pyod.utils.example import visualize TODO cannot import name 'njit'

In [6]:
df_operations = pd.read_csv(UtilsKy.GRAFANA_OPERATIONS 
                           , dtype=str, usecols = ['id', 'amount', 'pan', 'created', 'type', 'project_id'])

In [7]:
df = df_operations[df_operations.type == 'charge'].copy()

In [8]:
df['pan'] = df['pan'].str[:6]

In [9]:
min_date = min(df.created)

In [10]:
min_date

'2020-01-01 00:00:10'

In [11]:
max_date = max(df.created)

In [12]:
max_date

'2020-03-25 23:59:57'

In [13]:
from datetime import timedelta, datetime

In [14]:
date_start_test = datetime.strptime(max_date, '%Y-%m-%d %H:%M:%S') - timedelta(days=14)

In [15]:
start_test = date_start_test.strftime('%Y-%m-%d %H:%M:%S')
start_test

'2020-03-11 23:59:57'

In [16]:
teach =  df[(df.created < start_test)].copy()
test = df[(df.created >= start_test)].copy()

In [17]:
teach = teach.sample(n=100000, random_state=1)
test = test.sample(n=100000, random_state=1)

In [18]:
col_factors =['amount', 'pan']
teach[col_factors] = teach[col_factors].apply(pd.to_numeric, errors="coerce")
test[col_factors] = test[col_factors].apply(pd.to_numeric, errors="coerce")

In [19]:
st_pid = teach.groupby('project_id').amount.agg('mean').to_frame()

In [20]:
st_pid.columns

Index(['amount'], dtype='object')

In [21]:
st_pid.reset_index(level=0, inplace=True)

In [22]:
st_pid.columns = ["project_id", 'mean_amount_in_pid']

In [23]:
teach = pd.merge(teach, st_pid, how = 'left', on = ['project_id']).copy()
test = pd.merge(test, st_pid, how = 'left', on = ['project_id']).copy()

In [24]:
teach['average_amount_in_pid'] = teach.amount/ teach.mean_amount_in_pid
test['average_amount_in_pid'] = test.amount/ test.mean_amount_in_pid

In [25]:
test.head()

,id,type,amount,project_id,created,pan,mean_amount_in_pid,average_amount_in_pid
0,136481665,charge,1581.0,224f3d3120309f994e744af6ed64dddc18a82888,2020-03-22 00:43:13,516949,448.710182,3.523432
1,136498246,charge,500.0,8901f0b3f6f78a91dd7837199883454456f2ae37,2020-03-22 05:58:00,485320,94.430482,5.294900
2,136191950,charge,41.0,224f3d3120309f994e744af6ed64dddc18a82888,2020-03-19 17:12:07,516773,448.710182,0.091373
3,136628661,charge,53.0,224f3d3120309f994e744af6ed64dddc18a82888,2020-03-23 10:26:50,516837,448.710182,0.118116
4,135222314,charge,288.0,224f3d3120309f994e744af6ed64dddc18a82888,2020-03-12 21:36:42,516875,448.710182,0.641840


In [26]:
teach.head()

,id,type,amount,project_id,created,pan,mean_amount_in_pid,average_amount_in_pid
0,134254234,charge,141.00,224f3d3120309f994e744af6ed64dddc18a82888,2020-03-06 10:25:52,414949,448.710182,0.314234
1,130036244,charge,200.00,51dfe45c685dbb9ac1b650122838facfb4f209f1,2020-02-07 02:41:35,489504,141.005429,1.418385
2,125652873,charge,26.48,8901f0b3f6f78a91dd7837199883454456f2ae37,2020-01-12 02:21:09,402944,94.430482,0.280418
3,124716124,charge,31.00,224f3d3120309f994e744af6ed64dddc18a82888,2020-01-07 06:43:07,411997,448.710182,0.069087
4,128324238,charge,58.00,224f3d3120309f994e744af6ed64dddc18a82888,2020-01-26 15:33:34,516922,448.710182,0.129259


In [27]:
col_factors .extend(['mean_amount_in_pid', 'average_amount_in_pid'])

In [28]:
col_factors

['amount', 'pan', 'mean_amount_in_pid', 'average_amount_in_pid']

In [41]:
replace_val = -9999

db_teach = teach[col_factors].fillna(replace_val)
db_test = test[col_factors].fillna(replace_val)

teach_d = db_teach.values
test_d = db_test.values

In [44]:
# train kNN detector
clf_name = 'KNN'
clf = KNN()
# clf = KNN(n_neighbors=5, method='largest')
clf.fit(teach_d)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

In [45]:
test['pred'] = clf.predict(test_d)  # outlier labels (0 or 1)
test['scores'] = clf.decision_function(test_d)  # outlier scores

In [46]:
test_anomaly = test[test.pred == 1]